In [20]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [21]:
import os

cache_dir = "/root/autodl-fs"
os.environ["TRANSFORMERS_CACHE"] = cache_dir

[codecarbon INFO @ 17:30:35] Energy consumed for RAM : 0.038404 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:30:35] Energy consumed for RAM : 0.076678 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:30:35] Energy consumed for all GPUs : 0.048224 kWh. Total GPU Power : 29.276632021235223 W
[codecarbon INFO @ 17:30:35] Energy consumed for all CPUs : 0.008758 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:30:35] Energy consumed for all GPUs : 0.070079 kWh. Total GPU Power : 29.275998064765425 W
[codecarbon INFO @ 17:30:35] 0.095386 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:30:35] Energy consumed for all CPUs : 0.017379 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:30:35] 0.164135 kWh of electricity used since the beginning.


In [30]:
import wandb

# initiate W&B
wandb.init(project="huggingface", name="falcon_7b_threeturn")

[codecarbon INFO @ 17:33:46] Energy consumed for RAM : 0.007268 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:33:46] Energy consumed for all GPUs : 0.003731 kWh. Total GPU Power : 25.85958203099013 W
[codecarbon INFO @ 17:33:46] Energy consumed for all CPUs : 0.001636 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:33:46] 0.012634 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:33:50] Energy consumed for RAM : 0.048569 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:33:50] Energy consumed for all GPUs : 0.052762 kWh. Total GPU Power : 25.73112598996028 W
[codecarbon INFO @ 17:33:50] Energy consumed for all CPUs : 0.011053 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:33:50] 0.112384 kWh of electricity used since the beginning.
wandb: ERROR Error while calling W&B API: run 2vnj9mlj was previously created and deleted; try a new run name (<Response [409]>)
[codecarbon INFO @ 17:33:50] Energy consumed for RAM : 0.086839 kWh. RAM Power 

Problem at: /tmp/ipykernel_949/574286645.py 4 <module>


Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2313, in _atexit_cleanup
    self._on_finish()
  File "/root/miniconda3/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2561, in _on_finish
    _ = exit_handle.wait(timeout=-1, on_progress=self._on_progress_exit)
  File "/root/miniconda3/lib/python3.10/site-packages/wandb/sdk/lib/mailbox.py", line 298, in wait
    on_probe(probe_handle)
  File "/root/miniconda3/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2525, in _on_probe_exit
    result = handle.wait(timeout=0, release=False)
  File "/root/miniconda3/lib/python3.10/site-packages/wandb/sdk/lib/mailbox.py", line 281, in wait
    raise MailboxError("transport failed")
wandb.sdk.lib.mailbox.MailboxError: transport failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.10/site-packages/wandb/sdk/wandb_in

Error: An unexpected error occurred

[codecarbon INFO @ 17:34:01] Energy consumed for RAM : 0.008054 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:34:01] Energy consumed for all GPUs : 0.003838 kWh. Total GPU Power : 25.79416099203539 W
[codecarbon INFO @ 17:34:01] Energy consumed for all CPUs : 0.001813 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:34:01] 0.013706 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:34:05] Energy consumed for RAM : 0.049356 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:34:05] Energy consumed for all GPUs : 0.052870 kWh. Total GPU Power : 25.830643343375602 W
[codecarbon INFO @ 17:34:05] Energy consumed for all CPUs : 0.011230 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:34:05] 0.113456 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:34:05] Energy consumed for RAM : 0.087626 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:34:05] Energy consumed for all GPUs : 0.074725 kWh. Total GPU Power : 25.8236415989

## Finetune Falcon-7b on a Google colab

Welcome to this Google Colab notebook that shows how to fine-tune the recent Falcon-7b model on a single Google colab and turn it into a chatbot

We will leverage PEFT library from Hugging Face ecosystem, as well as QLoRA for more memory efficient finetuning

## Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [6]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

## Dataset

For our experiment, we will use the Guanaco dataset, which is a clean subset of the OpenAssistant dataset adapted to train general purpose chatbots.

The dataset can be found [here](https://huggingface.co/datasets/timdettmers/openassistant-guanaco)

In [22]:
from datasets import load_dataset

dataset_name = "JiazhenLiu01/SFT_LLAMA_362_threeturn"
dataset = load_dataset(dataset_name, split="train")

[codecarbon INFO @ 17:30:50] Energy consumed for RAM : 0.039191 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:30:50] Energy consumed for all GPUs : 0.048341 kWh. Total GPU Power : 28.27235156329821 W
[codecarbon INFO @ 17:30:50] Energy consumed for RAM : 0.077464 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:30:50] Energy consumed for all CPUs : 0.008935 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:30:50] 0.096467 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:30:50] Energy consumed for all GPUs : 0.070196 kWh. Total GPU Power : 28.2673109745727 W
[codecarbon INFO @ 17:30:50] Energy consumed for all CPUs : 0.017556 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:30:50] 0.165217 kWh of electricity used since the beginning.
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


## Loading the model

In this section we will load the [Falcon 7B model](https://huggingface.co/tiiuae/falcon-7b), quantize it in 4bit and attach LoRA adapters on it. Let's get started!

In [23]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    cache_dir=cache_dir
)
model.config.use_cache = False

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance. Therefore we will add `dense`, `dense_h_to_4_h` and `dense_4h_to_h` layers in the target modules in addition to the mixed query key value layer.

In [25]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

[codecarbon INFO @ 17:31:05] Energy consumed for RAM : 0.039978 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:31:05] Energy consumed for RAM : 0.078251 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:31:05] Energy consumed for all GPUs : 0.048592 kWh. Total GPU Power : 60.04316551537171 W
[codecarbon INFO @ 17:31:05] Energy consumed for all CPUs : 0.009112 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:31:05] Energy consumed for all GPUs : 0.070447 kWh. Total GPU Power : 60.048025462756776 W
[codecarbon INFO @ 17:31:05] 0.097681 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:31:05] Energy consumed for all CPUs : 0.017733 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:31:05] 0.166430 kWh of electricity used since the beginning.


## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [26]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

Then finally pass everthing to the trainer

In [27]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
[codecarbon INFO @ 17:31:15] [setup] RAM Tracking...
[codecarbon INFO @ 17:31:15] [setup] GPU Tracking...
[codecarbon INFO @ 17:31:15] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:31:15] [setup] CPU Tracking...
[codecarbon WARNING @ 17:31:15] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:31:17] We saw that you have a AMD EPYC 7T83 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 17:31:17] CPU Model on constant consumption mode: AMD EPYC 7T83 64-Core Processor
[codecarbon INFO @ 17:31:17] >>> Tracker's metadata:
[codecarbon INFO @ 17:31:17]   Platform system: Linux-5.4.0-153-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 17:31:17]   Python version: 

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [28]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [31]:
trainer.train()

[codecarbon WARNING @ 17:34:57] Already started tracking
[codecarbon INFO @ 17:35:01] Energy consumed for RAM : 0.011201 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:35:01] Energy consumed for all GPUs : 0.004611 kWh. Total GPU Power : 109.03469569836477 W
[codecarbon INFO @ 17:35:01] Energy consumed for all CPUs : 0.002521 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:35:01] 0.018334 kWh of electricity used since the beginning.


Step,Training Loss


[codecarbon INFO @ 17:35:05] Energy consumed for RAM : 0.052502 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:35:05] Energy consumed for all GPUs : 0.053959 kWh. Total GPU Power : 184.86591123113834 W
[codecarbon INFO @ 17:35:05] Energy consumed for all CPUs : 0.011938 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:35:05] 0.118399 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:35:05] Energy consumed for RAM : 0.090772 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:35:05] Energy consumed for all GPUs : 0.075821 kWh. Total GPU Power : 186.4546399978213 W
[codecarbon INFO @ 17:35:05] Energy consumed for all CPUs : 0.020560 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:35:05] 0.187153 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:35:16] Energy consumed for RAM : 0.011987 kWh. RAM Power : 188.85086917877197 W
[codecarbon INFO @ 17:35:16] Energy consumed for all GPUs : 0.006020 kWh. Total GPU Power : 338.078407682

BrokenPipeError: [Errno 32] Broken pipe

During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [27]:
trainer.push_to_hub("JiazhenLiu01/falcon-7b-oneturn")

adapter_model.safetensors:   0%|          | 0.00/522M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1715773408.autodl-container-e434419765-a9a1b7f6.7893.0:   0%|          | 0.00/16.2k [00:00…

CommitInfo(commit_url='https://huggingface.co/JiazhenLiu01/results/commit/fdedd952a44d4c5f8dfaf5ae22289b4b97289119', commit_message='JiazhenLiu01/falcon-7b-oneturn', commit_description='', oid='fdedd952a44d4c5f8dfaf5ae22289b4b97289119', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
trainer.save_model('falcon-7b-oneturn')

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
